# Защита персональных данных клиентов

**Цели:**
* Нужно защитить данные клиентов страховой компании.
* Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.
* Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
pd.options.display.float_format = '{:0.3f}'.format
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

## Загрузка данных

In [3]:
url = 'https://code.s3.yandex.net/datasets/'

try:
    df = pd.read_csv(url + 'insurance.csv')
    display(df.head())
except:
    print('Не удалось загрузить файл, проверьте путь.')

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.000,49600.000,1,0
1,0,46.000,38000.000,1,1
2,0,29.000,21000.000,0,0
3,0,21.000,41700.000,2,0
4,1,28.000,26100.000,0,0


Проверим наличие пропусков в данных.

In [4]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Узнаем сколько уникальных значений в целевой переменной.

In [5]:
df['Страховые выплаты'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

## Выводы:

* Выполнена выгрузка данных.
* Пропуски отсутствуют.
* Возможно актуальнее было бы решать задачу классификации.

# Применение обратимых матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$\begin{aligned}
a = Xw
\end{aligned}$

Задача обучения:

$\begin{aligned}
w = \arg\min_w MSE(Xw, y)
\end{aligned}$

Формула обучения:

$\begin{aligned}
w = (X^T X)^{-1} X^T y
\end{aligned}$

Для примера рассмотрим как будет вести себя RandomForestRegressor.

In [6]:
pipe_ll = make_pipeline(StandardScaler(), LinearRegression())
pipe_rf = make_pipeline(StandardScaler(), RandomForestRegressor())

Зададим обратимую матрицу A.

In [7]:
A = np.random.randint(100,size=(4, 4))
print(np.linalg.det(A), end='\n\n')
print(A @ np.linalg.inv(A))

-11334711.000000024

[[1.000 -0.000 -0.000 0.000]
 [-0.000 1.000 -0.000 -0.000]
 [-0.000 -0.000 1.000 0.000]
 [-0.000 0.000 0.000 1.000]]


Разделим выборку на тренировочную и тестовую.

In [8]:
features = df.drop('Страховые выплаты', axis=1).values
target = df['Страховые выплаты'].values
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.25, random_state=38)

y_hat_1 = pipe_ll.fit(X_train, y_train).predict(X_test)
y_hat_rf_1 = pipe_rf.fit(X_train, y_train).predict(X_test)

Для второго варианта будем применять признаки, умноженные на обратимую матрицу.

In [9]:
features_inv = (A @ features.T).T
print(features_inv.shape, end='\n\n')
print(features_inv)

(5000, 4)

[[645583.000 1044542.000 3821472.000 793954.000]
 [494788.000 801216.000 2928480.000 608345.000]
 [273435.000 443001.000 1618508.000 336203.000]
 ...
 [441196.000 713364.000 2611516.000 542586.000]
 [425794.000 688415.000 2519360.000 523467.000]
 [528388.000 854645.000 3127796.000 649863.000]]


In [10]:
features_inv = (A @ features.T).T
features_inv.shape
X_train, X_test, y_train, y_test = train_test_split(
    features_inv, target, test_size=0.25, random_state=38)

y_hat_2 = pipe_ll.fit(X_train, y_train).predict(X_test)
y_hat_rf_2 = pipe_rf.fit(X_train, y_train).predict(X_test)

Найдем MSE между предсказанными результатами.

In [11]:
mean_squared_error(y_hat_1, y_hat_2)

8.134282713704109e-27

Округлим предсказанные значения, так как страховые выплаты не могут быть дробными.

In [12]:
mean_squared_error(np.round(y_hat_1), np.round(y_hat_2))

0.0

Для случайного леса ошибка получилась выше.

In [13]:
mean_squared_error(y_hat_rf_1, y_hat_rf_2)

0.20902367999999996

In [14]:
mean_squared_error(np.round(y_hat_rf_1), np.round(y_hat_rf_2))

0.2304

**Обоснование:** 


$a_1 = Xw_1$

$w_1 = (X^T X)^{-1} X^T y$

$a_2 = (A \cdot X^T)^T w_2$

$w_2 = ((A \cdot X^T) (A \cdot X^T)^T)^{-1} (A \cdot X^T) y$

$ X \cdot (X^T X)^{-1} X^T y = (A \cdot X^T)^T \cdot ((A \cdot X^T) (A \cdot X^T)^T)^{-1} (A \cdot X^T) y$

Для проверки решим простую задачу из которой видно равенство получаемых $a_1$ и $a_2$.

In [15]:
X = np.array([[ 2,  5],
              [ 7,  4],
              [22,  9]])
y = np.array([[ 5],
              [8],
              [19]])

In [16]:
A = np.random.randint(100, size=(2, 2))

In [17]:
w = np.linalg.inv(X.T @ X) @ X.T @ y
X @ w

array([[5.250],
       [7.082],
       [19.270]])

In [18]:
X_inv = (A @ X.T).T
w_inv = np.linalg.inv(X_inv.T @ X_inv) @ X_inv.T @ y
X_inv @ w_inv

array([[5.250],
       [7.082],
       [19.270]])

Коэффициенты линейной регрессии имеют идеальную положительную корреляцию.

In [19]:
np.corrcoef(w.reshape(-1,),
            w_inv.reshape(-1,))

array([[1.000, 1.000],
       [1.000, 1.000]])

In [20]:
linregress_res = linregress(w_inv.reshape(-1,),
                            w.reshape(-1,))

Следовательно:

$ w = A \cdot w_{inv} + B$

In [21]:
X @ (w_inv * linregress_res.slope + linregress_res.intercept)

array([[5.250],
       [7.082],
       [19.270]])

## Выводы:

* Ошибка MSE фактически равна нулю.
* Можно считать, что умножение признаков на обратимую матрицу не влияет на качество предсказаний, получаемых линейной регрессии.
* Качество предсказания линейной модели не меняется из-за сильной линейной взаимосвязи.

# Алгоритм преобразования

1. Создать обратимую матрицу.
2. Проверить определитель матрицы. Если равен 0 - пересоздать матрицу.
3. Преобразовать признаки.

In [22]:
def feature_transform(X, A='None'):
    if type(A) != str:
        return (A @ X.T).T
    
    while(True):
        A = np.random.randint(100, size=(X.shape[1], X.shape[1]))
        if np.linalg.det(A) != 0:
            break
    return (A @ X.T).T, A

# Проверка алгоритма

Создадим свой регрессор.

In [23]:
class CustomRegressor:
    def __init__(self):
        return None
    
    def X_tranform(self, X):
        return np.concatenate((np.ones([X.shape[0], 1]),
                               np.array(X)), axis=1)
    
    def fit(self, X, y):
        X = self.X_tranform(X)
        y = np.array(y)
        self.w = np.linalg.inv(X.T @ X) @ X.T @ y
        return self

    def predict(self, X):
        X = self.X_tranform(X)
        return X @ self.w
    
pipe = make_pipeline(StandardScaler(),
                     CustomRegressor())

Заново разделим датафрейм на features и target.

In [24]:
features = df.drop('Страховые выплаты', axis=1).values
target = df['Страховые выплаты'].values
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.25, random_state=38)

X_train_transformed, A = feature_transform(X_train)

Измерим R2 для случая с и без преобразования.

In [25]:
r2_score(y_test,
         pipe.fit(X_train, y_train)
         .predict(X_test))

0.4362964410800213

In [26]:
r2_score(y_test,
         pipe.fit(X_train_transformed, y_train)
         .predict(feature_transform(X_test, A)))

0.4362964415264372

# Выводы

* Предложите алгоритм преобразования данных для решения задачи.
* Проведено сравнение качества моделей до и после преобразования.